In [1]:
import pycosat
import secrets
import math
import numpy as np
# import pandas
import pandas as pd
# import matplotlib
import matplotlib.pyplot as plt
# import seaborn
import seaborn as sns
from sklearn.neural_network import MLPRegressor
%matplotlib inline
NUM_FEATURES = 124


In [2]:
file = open("Scrum.dimacs")
lines = file.readlines()
names = []
for line in lines[:124]:
    names.append(line.split(' ')[2][:-1])

In [3]:
dimacs = lines[125:]
cnf = [[int(s)  for s in line.split(' ') if int(s) != 0] for line in dimacs]

In [4]:
cnfsol = []
i = 0
for sol in pycosat.itersolve(cnf):
    if i == 10000:
        break
    i += 1
    cnfsol.append(sol)
print(i)
solutions = [[1 if val > 0 else 0 for val in sol] for sol in cnfsol]

10000


In [5]:
costs = [secrets.randbelow(10) for _ in range(NUM_FEATURES)]
defective = [bool(secrets.randbelow(2)) for _ in range(NUM_FEATURES)]
used = [bool(secrets.randbelow(2)) for _ in range(NUM_FEATURES)]

In [6]:
#Equation = totalcost**2 + knowndefects**2 + (124 - featuresused)**2 + 1.5*(100 - userscore)**2

In [7]:
def fitness(individual, score):
    totalcost = sum(np.multiply(individual, costs))
    knowndefects = sum(np.multiply(individual, defective))
    featuresused = sum(np.multiply(individual, used))
    #print(totalcost, knowndefects, featuresused, score)
    sumsq = lambda *args: sum([i ** 2 for i in args])
    return sumsq(totalcost/10, knowndefects, 124-featuresused, 1.5*(100 - score))

In [8]:
def boolean_to_CNF(solution):
    cnf = []
    for i, val in enumerate(solution):
        if val == 1:
            cnf.append(i+1)
        else:
            cnf.append(-1*(i+1))
    return cnf

In [9]:
def validate_CNF(cnf, individual):
    for clause in cnf:
        valid = False
        for val in clause:
            if individual[abs(val)-1] == val:
                valid = True
        if not valid:
            return False
    return True

In [18]:
def can_we_stop_fucker(best_score, scores):
    if len(scores):
        print(best_score / max(scores))
        return best_score / max(scores) <= .30
    return False

In [11]:
def predict(item, X_train, y_train):
    #print(X_train, y_train)
    #import pdb;pdb.set_trace()
    clf = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
    return clf.predict([item])

In [12]:
def mutate(population, p=.1):
    mutated = list(map(lambda s: [1 - x for x in s if np.random.rand() <= p], population))
    return list(filter(lambda s: np.random.rand() <= .75, mutated))

In [13]:
def cull(population, scores):
    size = len(population)
    wanted_size = int(.25 * size)
    idx = np.argsort(scores)[:wanted_size]
    return population[idx], scores[idx]

In [14]:
def oracle(item):
    return np.random.randint(0,101)

In [15]:
def ga_method(initial_population, cnf, interaction_number, stop_criteria):
    cur_interaction_number = interaction_number
    cur_population = initial_population
    scores = []
    human_scores = []
    best_score = 1e7
    produced_items = []
    best_item = None
    cur_i = 0
    while not can_we_stop_fucker(best_score, scores):
        print(best_score)
        if cur_i == len(cur_population) -1:
            cur_population = mutate(produced_items)
            produced_items , scores = cull(produced_items, scores)
            cur_interaction_number = interaction_number
            cur_i = 0
            pass
        elif cur_interaction_number > 0:
            cur_interaction_number -= 1
            score = oracle(cur_population[cur_i])
            produced_items.append(cur_population[cur_i])
            ind_fit = fitness(cur_population[cur_i], score)
            scores.append(ind_fit)
            human_scores.append(score)
            if ind_fit < best_score:
                best_score = ind_fit
                best_item = cur_population[cur_i]
        elif cur_interaction_number == 0:
            score = predict(cur_population[cur_i], produced_items, human_scores)
            produced_items.append(cur_population[cur_i])
            ind_fit = fitness(cur_population[cur_i], score)
            scores.append(ind_fit)
            human_scores.append(score)
            if ind_fit < best_score:
                best_score = ind_fit
                best_item = cur_population[cur_i]
        cur_i+=1
            
        
    return best_item, best_score, produced_items, scores
    
    

In [ ]:
best_item, score, produced_items, scores = ga_method(solutions, cnf, 1000, 65000)

10000000.0
1.0
10364.24
0.5669811687178502
10364.24
0.5669811687178502
10364.24
0.5669811687178502
10364.24
0.47503162526354387
10364.24
0.4679571913099276
10209.89
0.4679571913099276
10209.89
0.385746713183461
10209.89
0.385746713183461
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89
0.33561384466401106
10209.89

0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
0.3006585769353208
9982.89
